In [62]:
## adding dependencies
import json
import requests
from pprint import pprint
import pandas as pd
from sqlalchemy import create_engine
import pymongo
from pandas.io.json import json_normalize
import hashlib
import time


In [66]:

m = hashlib.md5()
ts = str(time.time())
ts_byte = bytes(ts, 'utf-8')
m.update(ts_byte)
m.update(b"b999c2120532176b4609989cdd0b56242036cc97")
m.update(b"b2aeb1c91ad82792e4583eb08509f87a")
         

hasht = m.hexdigest()
hasht


'7e99aba51998e19490b9acbf4d963fca'

In [67]:
##base might be https://gateway.marvel.com:443
base_url = "https://gateway.marvel.com"
api_key = "b2aeb1c91ad82792e4583eb08509f87a"
query = "/v1/public/events" +"?"

query_url = base_url + query +"ts=" + ts+ "&apikey=" + api_key + "&hash=" + hasht
query_url

'https://gateway.marvel.com/v1/public/events?ts=1560614764.89988&apikey=b2aeb1c91ad82792e4583eb08509f87a&hash=7e99aba51998e19490b9acbf4d963fca'

In [68]:
data = requests.get(query_url).json()
data

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [ ]:
# Items of interest are the Title, description, start, end,  and all characters

In [37]:
eventTitle = []
eventDesc =[]
eventStart = []
eventEnd = []
eventCharacters =[]

for x in data["data"]["results"]:
    eTitle = x["title"]
    eventTitle.append(eTitle)
    eDesc = x["description"]
    eventDesc.append(eDesc)
    eStart = x["start"]
    eventStart.append(eStart)
    eEnd = x["end"]
    eventEnd.append(eEnd)
    eCharacters = x["characters"]
    eventCharacters.append(eCharacters)
    
    
    
Eventdf = pd.DataFrame()
Eventdf["Name"] = eventTitle
Eventdf["Desc"] = eventDesc
Eventdf["Start_Date"] = eventStart
Eventdf["End_Date"] = eventEnd
Eventdf["Characters"] = eventCharacters

Eventdf.head()


,Name,Desc,Start_Date,End_Date,Characters
0,Acts of Vengeance!,Loki sets about convincing the super-villains ...,1989-12-10 00:00:00,2008-01-04 00:00:00,"{'available': 102, 'collectionURI': 'http://ga..."
1,Age of Apocalypse,"In a twisted version of the world they knew, t...",1995-03-01 00:00:00,1996-06-01 00:00:00,"{'available': 91, 'collectionURI': 'http://gat..."
2,Age of Ultron,In a 10-issue event written by Brian Michael B...,2013-03-06 00:00:00,2013-06-19 00:00:00,"{'available': 13, 'collectionURI': 'http://gat..."
3,Age of X,In a strange reality where the X-Men never cam...,2011-01-26 00:00:00,2011-05-01 00:00:00,"{'available': 45, 'collectionURI': 'http://gat..."
4,All-New All-Different Marvel,"This fall, prepare for an all-new, all-differe...",None,None,"{'available': 0, 'collectionURI': 'http://gate..."


In [38]:
#Connecting the MongoDatabase
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.Marvel_events

In [39]:
#Creating the database Marvel_events
collection = db.Event_Details
df = Eventdf
records_ = df.to_dict(orient = 'records')
result = db.Event_Details.insert_many(records_)
result


